In [1]:
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Load Breast Cancer dataset

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

breast_cancer = load_breast_cancer()

breast_cancer_data = breast_cancer.data
breast_cancer_target = breast_cancer.target.reshape((breast_cancer.target.shape[0], 1))
# Binary One-hot Encoding
breast_cancer_target = np.array([[0.,1.] if ele==[1] else [1.,0.] for ele in breast_cancer_target])

x_train, x_test, y_train, y_test = train_test_split(
    breast_cancer_data,
    breast_cancer_target,
    train_size=0.8,
    random_state=1
)

In [3]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

((455, 30), (455, 2), (114, 30), (114, 2))


In [4]:
# Params
num_epochs = 50000
num_features = x_train.shape[1]

### Logistic Regression Model

In [5]:
x = tf.placeholder(tf.float32, shape=[None, num_features])
y = tf.placeholder(tf.float32, shape=[None, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

In [6]:
W = tf.Variable(tf.random_normal([num_features, 2]))
b = tf.Variable(tf.random_normal([2]))

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
regression = tf.matmul(x, W) + b

In [9]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y_, 
        logits=regression
    )
)

In [30]:
optimizer = tf.train.AdamOptimizer(0.05).minimize(cross_entropy)

In [31]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(num_epochs):
    sess.run(optimizer, feed_dict={x: x_train, y_: y_train})
    if ((i+1)<10 or (i+1)%2000==0):
        print("Iteration: %s, cross entropy = %s" % 
              ((i+1), 
               cross_entropy.eval(feed_dict={x: x_train, y_: y_train})
              )
             )

Iteration: 1, cross entropy = 192.057
Iteration: 2, cross entropy = 97.4778
Iteration: 3, cross entropy = 33.2319
Iteration: 4, cross entropy = 21.947
Iteration: 5, cross entropy = 56.9302
Iteration: 6, cross entropy = 83.1425
Iteration: 7, cross entropy = 89.971
Iteration: 8, cross entropy = 81.7798
Iteration: 9, cross entropy = 62.7296
Iteration: 2000, cross entropy = 0.192401
Iteration: 4000, cross entropy = 0.150844
Iteration: 6000, cross entropy = 0.135201
Iteration: 8000, cross entropy = 0.125614
Iteration: 10000, cross entropy = 0.113907
Iteration: 12000, cross entropy = 0.109724
Iteration: 14000, cross entropy = 0.138837
Iteration: 16000, cross entropy = 0.0860124
Iteration: 18000, cross entropy = 0.0753474
Iteration: 20000, cross entropy = 0.067818
Iteration: 22000, cross entropy = 0.374407
Iteration: 24000, cross entropy = 0.113615
Iteration: 26000, cross entropy = 0.0787765
Iteration: 28000, cross entropy = 0.124623
Iteration: 30000, cross entropy = 0.0727541
Iteration: 3200

In [32]:
softmax_regression = tf.nn.softmax(regression)

In [33]:
correct_prediction = tf.equal(tf.argmax(softmax_regression, 1), tf.argmax(y_test, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100.

In [34]:
print("accuray: %s %%" % accuracy.eval(feed_dict={x:x_test, y:y_test}))

accuray: 96.4912 %


In [36]:
predictions = sess.run(tf.argmax(softmax_regression, 1), feed_dict={x: x_test})
labels = sess.run(tf.argmax(y_test, 1))
print("Confusion Matrix: ")
print(sess.run(tf.confusion_matrix(labels=labels, predictions=predictions)))

Confusion Matrix: 
[[39  3]
 [ 1 71]]
